In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer  # For converting text data into TF-IDF vectors
from sklearn.metrics.pairwise import cosine_similarity  # For computing cosine similarity between vectors
from scipy.spatial.distance import pdist, squareform  # For pairwise distance computations and converting to a square matrix
import pickle
import math

# pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [4]:
#---------- load ----------
with open("rating_df_final.pk", "rb") as f:
    rating_df = pickle.load(f)
with open("anime_df_final.pk", "rb") as f:
    anime_df = pickle.load(f)

In [4]:
anime_df.head()

,anime_name,anime_id,Genres,Score,Synopsis,Score_num
0,Fullmetal Alchemist: Brotherhood,5114,"Action, Military, Adventure, Comedy, Drama, Ma...",9.19,"""In order for something to be obtained, someth...",9.19
1,Shingeki no Kyojin: The Final Season,40028,"Action, Military, Mystery, Super Power, Drama,...",9.17,Gabi Braun and Falco Grice have been training ...,9.17
2,Steins;Gate,9253,"Thriller, Sci-Fi",9.11,The self-proclaimed mad scientist Rintarou Oka...,9.11
3,Hunter x Hunter (2011),11061,"Action, Adventure, Fantasy, Shounen, Super Power",9.1,Hunter x Hunter is set in a world where Hunter...,9.10
4,Shingeki no Kyojin Season 3 Part 2,38524,"Action, Drama, Fantasy, Military, Mystery, Sho...",9.1,Seeking to restore humanity's diminishing hope...,9.10


In [14]:
rating_df.head()

,user_id,anime_id,rating,anime_name
4511,36,6512,7,Nyan Koi!
4512,36,5958,7,Sora no Otoshimono
4513,36,6802,8,So Ra No Wo To
4514,36,17187,8,Koukaku Kidoutai Arise: Ghost in the Shell - B...
4515,36,16498,8,Shingeki no Kyojin


In [6]:
def recommend_by_jaccard(
    title,
    anime_df,
    top_n=10,
    precomputed=None  # None, DataFrame 
):
    """
    recommend_by_jaccard is a function that recommends similar anime using Jaccard similarity
    based on either Genres or Themes.

    :param title: str
        The anime title (anime_name) to base recommendations on.

    :param anime_df: pd.DataFrame
        DataFrame containing at least 'anime_name' and the selected feature column ('Genres' or 'Themes').

    :param top_n: int
        Number of top similar results to return.

    :param precomputed: np.ndarray or None
        Optional precomputed Jaccard distance array to avoid recalculating distances.

    :return: dict
        A dictionary with:
            { "top": pd.Series } — the top-N most similar anime and their similarity scores.
    """

    type = 'Genres'
    # Check if title exists in the dataset
    if title not in anime_df['anime_name'].values:
        raise ValueError(f"'{title}' not found in dataset.")

    def compute_jaccard(df, col):
        cross_tab = pd.crosstab(df['anime_name'], df[col])
        distances = pdist(cross_tab.values, metric='jaccard')
        similarity = 1 - squareform(distances)
        return pd.DataFrame(similarity, index=cross_tab.index, columns=cross_tab.index)
    
    def compute_jaccard_array(df, col, arr):
        cross_tab = pd.crosstab(df['anime_name'], df[col])
        distances = arr
        similarity = 1 - squareform(distances)
        return pd.DataFrame(similarity, index=cross_tab.index, columns=cross_tab.index)
        
    df_jaccard = anime_df[["anime_name", type]]


    sim = precomputed
    if sim is None:
        sim = compute_jaccard(df_jaccard, type)
        if title not in sim.index:
            raise ValueError(f"'{title}' not found in similarity data.")
        top = sim.loc[title].sort_values(ascending=False)[1:top_n+1]
        return {"top": top}
        
    else:
        sim_df = compute_jaccard_array(df_jaccard, type, sim)

        if title not in sim_df.index:
            raise ValueError(f"'{title}' not found in similarity data.")
        top = sim_df.loc[title].sort_values(ascending=False)[1:top_n+1]
        return {"top": top}


In [7]:
recommend_by_jaccard("One Punch Man", anime_df)

{'top': anime_name
 One Punch Man 2nd Season             1.0
 One Punch Man 2nd Season Specials    1.0
 One Punch Man Specials               1.0
 One Punch Man: Road to Hero          1.0
 Zoids Shinseiki/Zero                 0.0
 5-toubun no Hanayome                 0.0
 xxxHOLiC Shunmuki                    0.0
 3-gatsu no Lion                      0.0
 Zero no Tsukaima                     0.0
 Zero no Tsukaima F                   0.0
 Name: One Punch Man, dtype: float64}

In [8]:
def tf_id_rec(title, anime_df, top_n, precomputed=None):
    """
    tf_id_rec is a function that recommends the most similar anime 
    using TF-IDF cosine similarity based on the Synopsis field.

    :param title: str
        The anime title (anime_name) to base recommendations on.

    :param anime_df: pd.DataFrame
        DataFrame containing at least 'anime_name' and 'Synopsis' columns.

    :param top_n: int
        The number of top similar anime to return (excluding the anime itself).

    :param precomputed: np.ndarray or None
        Optional precomputed cosine similarity matrix. If provided, it will be used instead of recomputing.

    :return: dict
        A dictionary with:
            { "top": pd.Series } — the top-N most similar anime and their similarity scores.
    """
    
# 1) verify the given title actually exists in the dataset
    if title not in anime_df['anime_name'].values:
        raise ValueError("'{0}' not found in dataset.".format(title))

    # 2) select only the columns we need and drop any rows where Synopsis is missing
    df_content = anime_df[['anime_name', 'Synopsis']].dropna(subset=['Synopsis'])

    # 3) build the TF-IDF matrix over all synopses
    vectorizer = TfidfVectorizer(min_df=2, max_df=0.7, stop_words='english')
    tfidf_mat = vectorizer.fit_transform(df_content['Synopsis'])
    tfidf_df  = pd.DataFrame(
        tfidf_mat.toarray(),
        index=df_content['anime_name'],
        columns=vectorizer.get_feature_names_out()
    )

    # 4) if a full cosine-similarity matrix was provided, reuse it
    if precomputed is not None:
        sim_df = pd.DataFrame(
            precomputed,
            index=tfidf_df.index,
            columns=tfidf_df.index
        )
        sims = sim_df.loc[title].sort_values(ascending=False)
        return {"top": sims}

    # 5) otherwise compute similarity between the target and every other anime
    target_vec   = tfidf_df.loc[title].values.reshape(1, -1)
    other_df     = tfidf_df.drop(title, axis=0)
    scores       = cosine_similarity(target_vec, other_df.values)[0]
    result_series = pd.Series(scores, index=other_df.index)

    # 6) pick the top_n highest-scoring titles
    top_similar = result_series.sort_values(ascending=False).iloc[:top_n]

    return {"top": top_similar}


In [9]:
tf_id_rec("Grand Blue", anime_df)

TypeError: tf_id_rec() missing 1 required positional argument: 'top_n'

<h1>USER BASED CF<h1>

In [32]:
user2movie = rating_df.groupby('user_id')['anime_id'].apply(list).to_dict()
movie2user = rating_df.groupby('anime_id')['user_id'].apply(list).to_dict()
user_movie = zip(rating_df['user_id'], rating_df['anime_id'])
user_movie_rating = zip(user_movie, rating_df['rating'])
user_movie2rating = dict(user_movie_rating)

In [22]:
def compute_similarity_matrix(user2movie, user_movie2rating, user_avg, min_common):
    """
    compute_similarity_matrix is a function that precomputes the similarity for each pair of users in 
    the training set and saves the similarity scores in a dictionary.
    :param user2movie: a dictionary that maps each user to a list of movie_ids.
    :param user_movie2rating: a dictionary that maps each (user, movie) pair to a rating.
    :param user_avg: a dictionary of the average ratings for each user.
    :param min_common: the required minimum number of common movies between a pair of movies to be eligible
    for similarity calculation.
    :return: a nested dictionary where the key is a user and the value is a dictionary of the similarity 
    score between the key user and all the other users.
    """
    similarity_matrix = {}
    all_users = list(user2movie.keys())
    print(len(all_users))
    for i, user1 in enumerate(all_users):
        if user1 not in similarity_matrix:
            similarity_matrix[user1] = {}
    
        if (i + 1) % 100 == 0:
            print('{} users processed'.format(i + 1))

        for j in range(i + 1, len(all_users)):
            user2 = all_users[j]
            pearson_similarity = calculate_pearson_similarity(user1, user2, user2movie, user_movie2rating, user_avg, min_common)
            similarity_matrix[user1][user2] = pearson_similarity
            
            if user2 not in similarity_matrix:
                similarity_matrix[user2] = {}
            similarity_matrix[user2][user1] = pearson_similarity

    return similarity_matrix

In [6]:
# --------------------------------------------------------------
#  SAVE SIMILARITY MATRICES  —  plain nested-dict version
# --------------------------------------------------------------
import pickle, os, gzip

def save_pickle(obj, path, compress=True):
    """Helper: write `obj` to *path* (optionally gzipped)."""
    if compress:
        with gzip.open(path + ".gz", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"✓ saved {path}.gz  ({os.path.getsize(path+'.gz')/1_048_576:.2f} MB)")
    else:
        with open(path, "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"✓ saved {path}  ({os.path.getsize(path)/1_048_576:.2f} MB)")

# --- call it for both matrices --------------------------------
save_pickle(user_similarity, "user_similarity")   # → user_similarity.pkl.gz
save_pickle(item_similarity, "item_similarity")   # → item_similarity.pkl.gz


✓ saved user_similarity.gz  (845.34 MB)
✓ saved item_similarity.gz  (27.89 MB)


In [7]:
# ============================================================
#  FASTER CF PIPELINE  –  builds or loads top-K similarity and
#  returns RMSE in a fraction of the previous run-time.
# ============================================================
import numpy as np, pandas as pd, pickle, os
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

K_NEIGH = 50        # keep only strongest 50 edges per row
MIN_CO   = 5        # min common items/users to consider
SEED     = 42

# ------------------------------------------------------------
# 1. split
train_df, test_df = train_test_split(
    rating_df, test_size=0.2, stratify=rating_df["user_id"], random_state=SEED
)

# ------------------------------------------------------------
# 2. sparse utility builders
uid2idx = {u:i for i,u in enumerate(train_df["user_id"].unique())}
iid2idx = {i:j for j,i in enumerate(train_df["anime_id"].unique())}
idx2uid = {i:u for u,i in uid2idx.items()}
idx2iid = {j:i for j,i in iid2idx.items()}

rows = train_df["user_id"].map(uid2idx)
cols = train_df["anime_id"].map(iid2idx)
data = train_df["rating"].astype(float)
R     = csr_matrix((data, (rows, cols)), shape=(len(uid2idx), len(iid2idx)))

user_mean  = np.asarray(R.sum(1)).ravel() / (R != 0).sum(1).A1
item_mean  = np.asarray(R.sum(0)).ravel() / (R != 0).sum(0).A1
global_mean = data.mean()



In [11]:
from scipy.sparse import csr_matrix, vstack
import pickle, os, numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

K_NEIGH = 50   # keep strongest 50 links per row

def topk_cosine(mat, kind, fname):
    """
    Build (or load) a sparse top-K cosine-similarity matrix.
      • mat   : CSR user-item or item-user matrix
      • kind  : 'user' or 'item'  (for the progress‐bar label)
      • fname : pickle file to cache the result
    """
    if os.path.exists(fname):
        with open(fname, "rb") as f:
            return pickle.load(f)

    print(f"Computing {kind} cosine similarity …")
    full = cosine_similarity(mat, dense_output=False)   # very fast C-code

    rows_topk = []
    for i in tqdm(range(full.shape[0]), desc=f"top-{K_NEIGH} {kind}"):
        row = full.getrow(i).tocoo()
        if row.nnz > K_NEIGH:                              # prune heavy rows
            idx = row.data.argsort()[::-1][:K_NEIGH]
            rows_topk.append(
                csr_matrix(
                    (row.data[idx], (np.zeros_like(idx), row.col[idx])),
                    shape=(1, full.shape[1])
                )
            )
        else:
            rows_topk.append(row)

    topk = vstack(rows_topk).tocsr()                       # <-- fix: sparse.vstack
    with open(fname, "wb") as f:
        pickle.dump(topk, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"✓ saved {fname}")
    return topk

# -----------------------------------------------------------------
user_sim = topk_cosine(R,   "user", "user_sim.pkl")      # (U × U)
item_sim = topk_cosine(R.T, "item", "item_sim.pkl")      # (I × I)


Computing user cosine similarity …


top-50 user:   0%|          | 0/10000 [00:00<?, ?it/s]

✓ saved user_sim.pkl
Computing item cosine similarity …


top-50 item:   0%|          | 0/9672 [00:00<?, ?it/s]

✓ saved item_sim.pkl


In [12]:
# ------------------------------------------------------------
# 3. fast lookup helpers
def predict_user_cf(u, i):
    if (u not in uid2idx) or (i not in iid2idx):          # cold-start
        return global_mean
    ui, ii = uid2idx[u], iid2idx[i]
    sims   = user_sim.getrow(ui).tocoo()
    if sims.nnz == 0:                                     # isolated user
        return user_mean[ui]
    numer = 0.0; denom = 0.0
    for s, v_idx in zip(sims.data, sims.col):
        r_vi = R[v_idx, ii]
        if r_vi == 0: continue
        numer += s * (r_vi - user_mean[v_idx])
        denom += abs(s)
    return user_mean[ui] + numer/denom if denom else user_mean[ui]

def predict_item_cf(u, i):
    if (u not in uid2idx) or (i not in iid2idx):
        return global_mean
    ui, ii = uid2idx[u], iid2idx[i]
    sims   = item_sim.getrow(ii).tocoo()
    if sims.nnz == 0:
        return item_mean[ii]
    numer = 0.0; denom = 0.0
    for s, j_idx in zip(sims.data, sims.col):
        r_uj = R[ui, j_idx]
        if r_uj == 0: continue
        numer += s * (r_uj - item_mean[j_idx])
        denom += abs(s)
    return item_mean[ii] + numer/denom if denom else item_mean[ii]


In [14]:
# ------------------------------------------------------------
# 4.  PARALLEL RMSE  (threads ➜ faster on GIL-releasing code)
# ------------------------------------------------------------
from concurrent.futures import ThreadPoolExecutor, as_completed
import os, numpy as np
from math import sqrt

N_WORKERS = max(4, os.cpu_count() // 2)   # sensible default

def _square_error(row, predict_fn):       # helper for thread pool
    return (row.rating - predict_fn(row.user_id, row.anime_id))**2

def rmse_parallel(predict_fn, workers=N_WORKERS, batch=20_000):
    """
    Compute RMSE on test_df using multithreading.
      • workers : number of concurrent threads
      • batch   : submit the jobs in chunks to keep memory low
    """
    squared_errs = []

    with ThreadPoolExecutor(max_workers=workers) as pool:
        futures = []
        for idx, row in enumerate(test_df.itertuples()):
            futures.append(pool.submit(_square_error, row, predict_fn))

            # optional: flush in batches to avoid thousands of open futures
            if len(futures) == batch:
                for f in as_completed(futures):
                    squared_errs.append(f.result())
                futures.clear()

        # collect anything left in the final partial batch
        for f in as_completed(futures):
            squared_errs.append(f.result())

    return sqrt(np.mean(squared_errs))


print("User-CF  RMSE :", round(rmse_parallel(predict_user_cf), 4))
print("Item-CF  RMSE :", round(rmse_parallel(predict_item_cf), 4))


User-CF  RMSE : 1.3639
Item-CF  RMSE : 1.2099


In [23]:
# Compute user avg
def compute_user_average(user2movie, user_movie2rating):
    """
    compute_user_average is a function that calculates the average rating for each user in the dataset.
    :param user2movie: a dictionary that maps each user to a list of movie_ids.
    :param user_movie2rating: a dictionary that maps each (user, movie) pair to a rating.
    :return: a dictionary containing the average rating for each user where the key is the user and the
    value is the average rating.
    """
    user_avg = {}
    for user, movies in user2movie.items():
        ratings = [user_movie2rating[(user, movie)] for movie in movies]
        user_avg[user] = np.mean(ratings)
    return user_avg

In [54]:
def pretty_print_recs(recs, rating_df, max_lines=10):
    """
    Show a recommendation list in a readable table.

    Parameters
    ----------
    recs : list[tuple(int, float)]
        Output of a recommender, e.g.
        [(19, 10.0), (32, 9.7), …]  where
        rec[0] = anime_id   and   rec[1] = predicted / hybrid score.
    rating_df : pd.DataFrame
        The ratings table that already lives in memory.  
        Must contain the columns  ['anime_id', 'anime_name'].
        (Duplicates are fine – we just grab the first match.)
    max_lines : int, default 10
        How many rows to print.
    """

    # --- build a quick lookup:  anime_id → anime_name -------------
    id2name = (
        rating_df[["anime_id", "anime_name"]]
        .drop_duplicates("anime_id")
        .set_index("anime_id")["anime_name"]
        .to_dict()
    )

    # --- nicely formatted output ---------------------------------
    print("┌────────┬──────────────────────────────────────────────────┬────────┐")
    print("│ Rank   │ Anime title                                      │ Score  │")
    print("├────────┼──────────────────────────────────────────────────┼────────┤")

    for rank, (aid, score) in enumerate(recs[:max_lines], start=1):
        title = id2name.get(aid, f"<? unknown id {aid} ?>")
        print(f"│ {rank:>3}    │ {title[:50]:<50} │ {score:>6.2f} │")

    print("└────────┴──────────────────────────────────────────────────┴────────┘")


##Item-Based CF##

In [ ]:
# =============================================================
#  HYBRID COLLABORATIVE FILTER 
#  -------------------------------------------------------------
#  · user-based CF  (top-K cosine on USER × ITEM matrix)
#  · item-based CF  (top-K cosine on ITEM × USER matrix)
#  · HybridCF = α·userCF + (1-α)·itemCF
# =============================================================

import pandas as pd, numpy as np, pickle, os
from scipy.sparse import csr_matrix, vstack
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

# ---------------- 1. helper dictionaries ---------------------
user2movie = rating_df.groupby("user_id")["anime_id"].apply(list).to_dict()
movie2user = rating_df.groupby("anime_id")["user_id"].apply(list).to_dict()
user_movie2rating = {(r.user_id, r.anime_id): r.rating for r in rating_df.itertuples()}



In [35]:
# ---------------- 2. sparse USER × ITEM matrix ---------------
uid2idx = {u:i for i,u in enumerate(rating_df["user_id"].unique())}
iid2idx = {i:j for j,i in enumerate(rating_df["anime_id"].unique())}
idx2iid = {j:i for i,j in iid2idx.items()}

rows = rating_df["user_id"].map(uid2idx)
cols = rating_df["anime_id"].map(iid2idx)
data = rating_df["rating"].astype(float)
R = csr_matrix((data, (rows, cols)), shape=(len(uid2idx), len(iid2idx)))

user_mean  = np.asarray(R.sum(1)).ravel() / (R != 0).sum(1).A1
item_mean  = np.asarray(R.sum(0)).ravel() / (R != 0).sum(0).A1
global_mean = data.mean()



In [36]:
# ---------------- 3. cosine similarity (top-K pruning) -------
def topk_cosine(mat, K=50):
    sim = cosine_similarity(mat, dense_output=False)          # CSR × CSR
    trimmed = []
    for i in tqdm(range(sim.shape[0]), desc=f"top-{K} prune"):
        row = sim.getrow(i).tocoo()
        idx = row.data.argsort()[::-1][:K]                    # keep best-K
        trimmed.append(
            csr_matrix((row.data[idx], (np.zeros_like(idx), row.col[idx])),
                       shape=(1, sim.shape[1]))
        )
    return vstack(trimmed).tocsr()

user_sim = topk_cosine(R,   K=50)      # (U × U) sparse
item_sim = topk_cosine(R.T, K=50)      # (I × I) sparse



top-50 prune:   0%|          | 0/10000 [00:00<?, ?it/s]

top-50 prune:   0%|          | 0/10015 [00:00<?, ?it/s]

In [37]:
# ---------------- 4. rating predictors -----------------------
def predict_user_cf(u, i, sim_cut=0.05):
    if (u not in uid2idx) or (i not in iid2idx): return global_mean
    ui, ii = uid2idx[u], iid2idx[i]
    sims   = user_sim.getrow(ui).tocoo()
    num=den=0.0
    for s,v in zip(sims.data, sims.col):
        if s < sim_cut: break
        r_vi = R[v, ii]
        if r_vi==0: continue
        num += s * (r_vi - user_mean[v])
        den += abs(s)
    return user_mean[ui] + num/den if den else user_mean[ui]

def predict_item_cf(u, i, sim_cut=0.05):
    if (u not in uid2idx) or (i not in iid2idx): return global_mean
    ui, ii = uid2idx[u], iid2idx[i]
    sims   = item_sim.getrow(ii).tocoo()
    num=den=0.0
    for s,j in zip(sims.data, sims.col):
        if s < sim_cut: break
        r_uj = R[ui, j]
        if r_uj==0: continue
        num += s * (r_uj - item_mean[j])
        den += abs(s)
    return item_mean[ii] + num/den if den else item_mean[ii]



In [38]:
# ---------------- 5. recommendation helpers -----------------
def uCF_list(user, top_n=500):
    seen = set(user2movie.get(user, []))
    cand = [m for m in movie2user if m not in seen]
    scores = {m: predict_user_cf(user, m) for m in cand}
    return sorted(scores.items(), key=lambda x:x[1], reverse=True)[:top_n]

def itemCF_list(user, top_n=500):
    seen = set(user2movie.get(user, []))
    cand = [m for m in movie2user if m not in seen]
    scores = {m: predict_item_cf(user, m) for m in cand}
    return sorted(scores.items(), key=lambda x:x[1], reverse=True)[:top_n]


In [39]:
# ---------------- 6. HYBRID recommender ----------------------
def HybridCF(user, top_n=10, alpha=0.6):
    u_preds = dict(uCF_list(user, top_n=1000))
    i_preds = dict(itemCF_list(user, top_n=1000))
    movies  = set(u_preds)|set(i_preds)
    hybrid  = {}
    for m in movies:
        if m in u_preds and m in i_preds:
            hybrid[m] = alpha*u_preds[m] + (1-alpha)*i_preds[m]
        elif m in u_preds:
            hybrid[m] = u_preds[m]
        else:
            hybrid[m] = i_preds[m]
    return sorted(hybrid.items(), key=lambda x:x[1], reverse=True)[:top_n]


In [41]:
# ------------ example run (using rating_df for the lookup) -----------
TARGET_USER = 36
print(f"\nHybrid recommendations for user {TARGET_USER}:\n")

# pre-build a fast ID→name map from rating_df
id2name = rating_df.drop_duplicates("anime_id") \
                   .set_index("anime_id")["anime_name"]  # Series

for mid, score in HybridCF(TARGET_USER, top_n=10, alpha=0.6):
    title = id2name.get(mid, f"(id {mid})")   # graceful fallback
    print(f"{title:<45}  ({score:.2f})")



Hybrid recommendations for user 36:

Kore ga UFO da! Soratobu Enban                 (17.86)
Futatsu no Kurumi                              (17.86)
Glass no Kamen: Sen no Kamen wo Motsu Shoujo   (16.36)
Penguin's Memory: Shiawase Monogatari          (16.11)
Moonfesta                                      (15.86)
True Tears Epilogue                            (15.49)
Genshiken Nidaime OVA                          (15.27)
Houkago Initiation                             (14.86)
Zetsubou Funsai Shoujo ∞ Amida                 (14.86)
Amada Anime Series: Super Mario Brothers       (14.06)


In [59]:
rec = uCF_list(36, top_n=10)

In [60]:
pretty_print_recs(rec, rating_df)

┌────────┬──────────────────────────────────────────────────┬────────┐
│ Rank   │ Anime title                                      │ Score  │
├────────┼──────────────────────────────────────────────────┼────────┤
│   1    │ Kimi to Boku. 2                                    │  12.39 │
│   2    │ Texhnolyze                                         │  11.99 │
│   3    │ Megalo Box                                         │  11.74 │
│   4    │ Kimi to Boku.                                      │  11.39 │
│   5    │ JoJo no Kimyou na Bouken Part 4: Diamond wa Kudake │  10.92 │
│   6    │ Yama no Susume: Second Season                      │  10.91 │
│   7    │ Yakusoku no Neverland                              │  10.91 │
│   8    │ Kaguya-sama wa Kokurasetai: Tensai-tachi no Renai  │  10.91 │
│   9    │ Kizumonogatari III: Reiketsu-hen                   │  10.84 │
│  10    │ Owarimonogatari 2nd Season                         │  10.84 │
└────────┴───────────────────────────────────────────────